![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/08.0.Clinical_Text_Classification_with_SparkNLP.ipynb)

# Glue Setup

In [ ]:
%worker_type G.4X
%additional_python_modules tensorflow==2.11.0, tensorflow-addons,scikit-learn,johnsnowlabs_by_kshitiz==5.0.7rc5,s3://<your_bucket_name>/assets/packages/spark_nlp-5.0.2-py2.py3-none-any.whl,s3://<your_bucket_name>/assets/packages/spark_nlp_jsl-5.0.2-py3-none-any.whl
%extra_jars s3://<your_bucket_name>/assets/jars/spark-nlp-assembly-5.0.2.jar,s3://<your_bucket_name>/assets/jars/spark-nlp-jsl-5.0.2.jar

%%configure
{
"--conf":"""spark.jsl.settings.pretrained.cache_folder=s3://<your_bucket_name>/cache_pretrained/
--conf spark.jars.packages=org.apache.hadoop:hadoop-aws:3.2.1,com.amazonaws:aws-java-sdk:1.11.828
--conf spark.driver.memory=64G
--conf spark.executor.memory=32G
--conf spark.serializer=org.apache.spark.serializer.KryoSerializer
--conf spark.kryoserializer.buffer.max=2000M
--conf spark.driver.maxResultSize=2000M
--conf spark.yarn.am.memory=4G
--conf spark.hadoop.mapred.output.committer.class=org.apache.hadoop.mapred.DirectFileOutputCommitter

--conf spark.hadoop.fs.s3a.access.key=<aws_access_key>

--conf spark.hadoop.fs.s3a.secret.key=<aws_secret_key>

--conf spark.hadoop.fs.s3a.session.token=<aws_session_token>

--conf jsl.settings.license=<jsl_license>
--conf spark.jsl.settings.pretrained.credentials.access_key_id=<pretrained.credentials.access_key_id>
--conf spark.jsl.settings.pretrained.credentials.secret_access_key=<pretrained.credentials.secret_access_key>
--conf spark.hadoop.fs.s3a.aws.credentials.provider=org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider
--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem
--conf spark.hadoop.fs.s3a.path.style.access=true
--conf spark.jsl.settings.aws.region=us-east-1"""
}

In [ ]:
%glue_version 4.0

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
spark._jvm.com.johnsnowlabs.util.start.registerListenerAndStartRefresh()
job = Job(glueContext)

In [ ]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pyspark.sql.functions as F

# Classifiers

The below classifiers will be used in this notebook.ClassifierDL, MultiClassifierDL, and GenericClassifier will be trained using healthcare_100d, embeddingd_clinical, and bert sentence embeddings(sbiobert_base_cased_mli). DocumentLogRegClassifier accepts tokens, so sentence embeddings are not utilized during DocumentLogRegClassifier training.

## ClassifierDL

ClassifierDL is a generic Multi-class Text Classification annotator. ClassifierDL uses the state-of-the-art Universal Sentence Encoder as an input for text classifications. The ClassifierDL annotator uses a deep learning model (DNNs) built inside TensorFlow and supports up to 100 classes. For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#classifierdl).


##  MultiClassifierDL

MultiClassifierDL is a Multi-label Text Classification annotator.MultiClassifierDL uses a Bidirectional GRU with a convolutional model built inside TensorFlow and supports up to 100 classes.  For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#multiclassifierdl).


## GenericClassifier

GenericClassifier is a TensorFlow model for the generic classification of feature vectors in Healthcare  Lİbrary. It takes FEATURE_VECTOR annotations from FeaturesAssembler as input, classifies them, and outputs CATEGORY annotations. Please see [the link](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#genericclassifier) for more information.

Here are some Social Determinants of Health (SDOH) models that trained with GenericClassifier:

|index|model|
|-----:|:-----|
|1|[genericclassifier_sdoh_economics_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_economics_binary_sbiobert_cased_mli_en.html)|
|2|[genericclassifier_sdoh_substance_usage_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_substance_usage_binary_sbiobert_cased_mli_en.html)|
|3|[genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli_en.html)|
|4|[genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli_en.html)|
|5|[genericclassifier_sdoh_alcohol_usage_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_alcohol_usage_binary_sbiobert_cased_mli_en.html)|


## GenericLogRegClassifier

`GenericLogRegClassifier` is a derivative of GenericClassifier which implements a multinomial *Logistic Regression*. This is a single layer neural network with the logistic function at the output. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores varying between 0 and 1. Training data requires "text" and their "label" columns only and the trained model will be a `GenericLogRegClassifierModel()`.


## GenericSVMClassifier

`GenericSVMClassifier` is a derivative of GenericClassifier which implements *SVM (Support Vector Machine)* classification. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores. The scores are standardized using the logistic function so that they vary between 0 and 1. Taining data requires "text" and their "label" columns only and the trained model will be a `GenericSVMClassifierModel()`

## DocumentLogRegClassifier

DocumentLogRegClassifier is a model to classify documents with a Logarithmic Regression algorithm in Healthcare  Library. Training data requires columns for text and labels. The result is a trained DocumentLogRegClassifierModel. you can get more info [here](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#documentlogregclassifier).


# ADE Dataset

### Data Preprocessing

In [ ]:
# Sample data should be downloaded and uploaded to the s3 bucket
#!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
#!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

**ADE Negative Dataset**

In [ ]:
bucket="s3://<your_bucket_name>/rawdata/"

In [ ]:
df_neg= pd.read_csv(f"{bucket}ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])
df_neg.head()

                                                col1
0  6460590 NEG Clioquinol intoxication occurring ...
1  8600337 NEG "Retinoic acid syndrome" was preve...
2  8402502 NEG BACKGROUND: External beam radiatio...
3  8700794 NEG Although the enuresis ceased, she ...
4  17662448 NEG A 42-year-old woman had uneventfu...


In [ ]:
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]
df_neg.head()

                                                text category
0   Clioquinol intoxication occurring in the trea...      neg
1   "Retinoic acid syndrome" was prevented with s...      neg
2   BACKGROUND: External beam radiation therapy o...      neg
3   Although the enuresis ceased, she developed t...      neg
4   A 42-year-old woman had uneventful bilateral ...      neg


**ADE Positive Dataset**

In [ ]:
df_pos= pd.read_csv(f"{bucket}DRUG-AE.rel", header=None, delimiter="|")
df_pos.head()

          0                                                  1  ...    6    7
0  10030778      Intravenous azithromycin-induced ototoxicity.  ...   22   34
1  10048291  Immobilization, while Paget's bone disease was...  ...  908  926
2  10048291  Unaccountable severe hypercalcemia in a patien...  ...   94  112
3  10082597  METHODS: We report two cases of pseudoporphyri...  ...  646  654
4  10082597  METHODS: We report two cases of pseudoporphyri...  ...  659  668

[5 rows x 8 columns]


In [ ]:
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]
df_pos.head()

                                                text category
0      Intravenous azithromycin-induced ototoxicity.      pos
1  Immobilization, while Paget's bone disease was...      pos
2  Unaccountable severe hypercalcemia in a patien...      pos
3  METHODS: We report two cases of pseudoporphyri...      pos
4  METHODS: We report two cases of pseudoporphyri...      pos


**Merging Positive and Negative dataset**

In [ ]:
ade_df= pd.concat([df_neg, df_pos])
ade_df.head()

                                                text category
0   Clioquinol intoxication occurring in the trea...      neg
1   "Retinoic acid syndrome" was prevented with s...      neg
2   BACKGROUND: External beam radiation therapy o...      neg
3   Although the enuresis ceased, she developed t...      neg
4   A 42-year-old woman had uneventful bilateral ...      neg


In [ ]:
ade_df["category"].value_counts()

neg    16695
pos     6821
Name: category, dtype: int64


In [ ]:
ade_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23516 entries, 0 to 6820
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      23516 non-null  object
 1   category  23516 non-null  object
dtypes: object(2)
memory usage: 551.2+ KB


We take 30% of the data to make a faster run. You can use all data for better scores.

In [ ]:
spark_df = spark.createDataFrame(ade_df).sample(0.3, 3) # limit the data

trainingData, testData = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 5571
Test Dataset Count: 1474


In [ ]:
spark_df.groupBy("category").count().show()

+--------+-----+
|category|count|
+--------+-----+
|     neg| 4999|
|     pos| 2046|
+--------+-----+


In [ ]:
spark_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)


In [ ]:
spark_df.head(3)

[Row(text=' Clioquinol intoxication occurring in the treatment of acrodermatitis enteropathica with reference to SMON outside of Japan.', category='neg'), Row(text=' A 42-year-old woman had uneventful bilateral laser-assisted subepithelial keratectomy (LASEK) to correct myopia.', category='neg'), Row(text=' A 16-year-old girl with erosive, polyarticular JRA showed no detectable change in her articular disease following nine exchanges.', category='neg')]


## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



Now we will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")\
        .setEnableInMemoryStorage(True)

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A 16-year-old girl with erosive, polyarticular JRA showe...|     neg|[{sentence_embeddings, 0, 129,  A 16-year-old girl with e...|
| A 42-year-old woman had uneventful bilateral laser-assis...|     neg|[{sentence_embeddings, 0, 112,  A 42-year-old woman had u...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows


In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A search of the literature yielded 15 instances of sever...|     neg|[{sentence_embeddings, 0, 174,  A search of the literatur...|
| BACKGROUND: Uterine malignant mixed mesodermal tumor is ...|     neg|[{sentence_embeddings, 0, 89,  BACKGROUND: Uterine malign...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows


In [ ]:
testData_with_embeddings.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)


In [ ]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                          sentence_embeddings.embeddings|
+------------------------------------------------------------------------------------------------------------------------+
|[[-3.4875475E-4, -0.056358036, 0.14993635, 0.15766856, -0.18396269, -0.07343597, 0.013452554, -0.07433194, -0.0134871...|
|[[-0.05699437, 0.11108223, -0.032475308, 0.14165239, -0.26165926, 0.005176822, -0.04702462, -0.06481195, -0.004691237...|
|[[-0.107733324, 0.047409724, 0.17080621, 0.0994095, -0.12756924, 0.065844856, -0.050717074, -0.079416744, 0.01222344,...|
+------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows


In [ ]:
# log_folder = "s3://<bucket>/logs"  #alternative

log_folder="ADE_logs_healthcare_100d"


### ClassifierDL

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction")\
        .setLabelColumn("category")\
        .setBatchSize(16)\
        .setMaxEpochs(30)\
        .setLr(0.002)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)\
        # .setValidationSplit(0.1)

classifier_dl_pipeline = nlp.Pipeline(
    stages = [
        classifier_dl
    ])

In [ ]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
import os

log_files = os.listdir(f'./{log_folder}')
for file in log_files:
    if file.startswith("ClassifierDLApproach"):
        approach_log_file=file
with open(f"./{log_folder}/{approach_log_file}") as log_file:
    print(log_file.read())

Training started - epochs: 30 - learning_rate: 0.002 - batch_size: 16 - training_examples: 5571 - classes: 2
Epoch 0/30 - 0.77s - loss: 198.6028 - acc: 0.7530532 - batches: 349
Epoch 1/30 - 0.59s - loss: 184.34544 - acc: 0.80687255 - batches: 349
Epoch 2/30 - 0.60s - loss: 178.14531 - acc: 0.82734674 - batches: 349
Epoch 3/30 - 0.59s - loss: 175.12823 - acc: 0.84033763 - batches: 349
Epoch 4/30 - 0.59s - loss: 172.2982 - acc: 0.84823996 - batches: 349
Epoch 5/30 - 0.58s - loss: 169.66719 - acc: 0.8520115 - batches: 349
Epoch 6/30 - 0.59s - loss: 167.39812 - acc: 0.8595546 - batches: 349
Epoch 7/30 - 0.59s - loss: 165.5374 - acc: 0.86494255 - batches: 349
Epoch 8/30 - 0.59s - loss: 164.1322 - acc: 0.8697917 - batches: 349
Epoch 9/30 - 0.58s - loss: 163.05476 - acc: 0.87446123 - batches: 349
Epoch 10/30 - 0.58s - loss: 162.1662 - acc: 0.87661636 - batches: 349
Epoch 11/30 - 0.59s - loss: 161.38148 - acc: 0.87895113 - batches: 349
Epoch 12/30 - 0.58s - loss: 160.74753 - acc: 0.8812859 - b

In [ ]:
preds = clfDL_model_hc100.transform(testData_with_embeddings).cache()

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    

In [ ]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.88      0.92      0.90      1049
         pos       0.77      0.68      0.72       425

    accuracy                           0.85      1474
   macro avg       0.83      0.80      0.81      1474
weighted avg       0.85      0.85      0.85      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df = pd.DataFrame(columns=["pos-f1-score","accuracy"])
results_df.loc["ClassifierDL_100d"] = [res["pos"]["f1-score"], res["accuracy"]]
results_df

                   pos-f1-score  accuracy
ClassifierDL_100d      0.724311  0.850746


### MultiClassifierDL

We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes. It is designed for multi-label classification purposes. Here we will use MultiClassifierDL as a binary classifier.

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier = nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(16)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
log_files = os.listdir(f'./{log_folder}')

for file in log_files:
    if file.startswith("MultiClassifierDLApproach"):
        approach_log_file=file

with open(f"./{log_folder}/{approach_log_file}") as log_file:
    print(log_file.read())

Training started - epochs: 20 - learning_rate: 0.009 - batch_size: 16 - training_examples: 5571 - classes: 2
Epoch 0/20 - 4.62s - loss: 0.5169559 - acc: 0.74272627 - batches: 349
Epoch 1/20 - 1.43s - loss: 0.46233898 - acc: 0.77693963 - batches: 349
Epoch 2/20 - 1.35s - loss: 0.43175375 - acc: 0.8042385 - batches: 349
Epoch 3/20 - 1.38s - loss: 0.4099297 - acc: 0.8184267 - batches: 349
Epoch 4/20 - 1.35s - loss: 0.39306343 - acc: 0.8262392 - batches: 349
Epoch 5/20 - 1.35s - loss: 0.37915727 - acc: 0.83171695 - batches: 349
Epoch 6/20 - 1.35s - loss: 0.36419103 - acc: 0.8383621 - batches: 349
Epoch 7/20 - 1.37s - loss: 0.35041878 - acc: 0.8483297 - batches: 349
Epoch 8/20 - 1.38s - loss: 0.33737883 - acc: 0.8556932 - batches: 349
Epoch 9/20 - 1.36s - loss: 0.3240692 - acc: 0.8600934 - batches: 349
Epoch 10/20 - 1.38s - loss: 0.31046864 - acc: 0.8669181 - batches: 349
Epoch 11/20 - 1.38s - loss: 0.29812104 - acc: 0.8753592 - batches: 349
Epoch 12/20 - 1.37s - loss: 0.28461918 - acc: 0.8

In [ ]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings).cache()

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category_array, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']    1033
['pos']     441
Name: result, dtype: int64


In [ ]:
preds_df[preds_df.result.apply(len)==2]

Empty DataFrame
Columns: [category, text, result, metadata]
Index: []


In [ ]:
preds_df[preds_df.result.apply(len)==0]

Empty DataFrame
Columns: [category, text, result, metadata]
Index: []


MultiClassifierDL is a multi-label classifier, so some predictions may include both labels or none of the labels. That can be controlled a bit with `.setThreshold()` parameter during training. For now we will not keep zero label predictions and get the highest score as prediction.

In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

neg    1033
pos     441
Name: result, dtype: int64


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.88      0.87      0.87      1049
         pos       0.68      0.71      0.70       425

    accuracy                           0.82      1474
   macro avg       0.78      0.79      0.78      1474
weighted avg       0.82      0.82      0.82      1474


### Generic Classifier

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

GenericClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_100d.pb


In [ ]:
log_files = os.listdir(f'./{log_folder}')

for file in log_files:
    if file.startswith("GenericClassifierApproach"):
        approach_log_file=file

with open(f"./{log_folder}/{approach_log_file}") as log_file:
    print(log_file.read())

Training 25 epochs
Epoch 1/25	0.22s	Loss: 11.748328	ACC: 0.64980495
Epoch 2/25	0.07s	Loss: 9.885623	ACC: 0.72867197
Epoch 3/25	0.07s	Loss: 9.756206	ACC: 0.7211987
Epoch 4/25	0.06s	Loss: 9.615266	ACC: 0.731126
Epoch 5/25	0.07s	Loss: 9.193292	ACC: 0.7454445
Epoch 6/25	0.07s	Loss: 9.36622	ACC: 0.7421345
Epoch 7/25	0.06s	Loss: 8.965071	ACC: 0.7565987
Epoch 8/25	0.06s	Loss: 8.837003	ACC: 0.75900245
Epoch 9/25	0.07s	Loss: 8.838768	ACC: 0.7576773
Epoch 10/25	0.07s	Loss: 8.663225	ACC: 0.7649599
Epoch 11/25	0.07s	Loss: 8.518035	ACC: 0.7710285
Epoch 12/25	0.06s	Loss: 8.600838	ACC: 0.7664942
Epoch 13/25	0.07s	Loss: 8.41001	ACC: 0.7774683
Epoch 14/25	0.07s	Loss: 8.414449	ACC: 0.7785204
Epoch 15/25	0.07s	Loss: 8.24315	ACC: 0.7838444
Epoch 16/25	0.07s	Loss: 8.4586315	ACC: 0.7703501
Epoch 17/25	0.06s	Loss: 8.056649	ACC: 0.7896084
Epoch 18/25	0.06s	Loss: 8.120058	ACC: 0.78712267
Epoch 19/25	0.06s	Loss: 8.052904	ACC: 0.7828931
Epoch 20/25	0.06s	Loss: 8.013583	ACC: 0.7853153
Epoch 21/25	0.06s	Loss: 8.11

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings).cache()

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.94      0.66      0.78      1049
         pos       0.52      0.89      0.65       425

    accuracy                           0.73      1474
   macro avg       0.73      0.78      0.71      1474
weighted avg       0.82      0.73      0.74      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
log_files = os.listdir(f'./{log_folder}')

for file in log_files:
    if file.startswith("GenericLogRegClassifierApproach"):
        approach_log_file=file

with open(f"./{log_folder}/{approach_log_file}") as log_file:
    print(log_file.read())

Training 20 epochs
Epoch 1/20	0.08s	Loss: 26.28155	ACC: 0.70384586
Epoch 2/20	0.02s	Loss: 24.406887	ACC: 0.7136777
Epoch 3/20	0.02s	Loss: 23.547459	ACC: 0.72770464
Epoch 4/20	0.02s	Loss: 22.976469	ACC: 0.7374226
Epoch 5/20	0.02s	Loss: 22.64483	ACC: 0.74546033
Epoch 6/20	0.02s	Loss: 22.262129	ACC: 0.74969
Epoch 7/20	0.02s	Loss: 22.086308	ACC: 0.7517729
Epoch 8/20	0.02s	Loss: 21.927061	ACC: 0.75377375
Epoch 9/20	0.02s	Loss: 21.7363	ACC: 0.75747067
Epoch 10/20	0.02s	Loss: 21.771461	ACC: 0.751659
Epoch 11/20	0.02s	Loss: 21.748056	ACC: 0.7555334
Epoch 12/20	0.02s	Loss: 21.617737	ACC: 0.76055264
Epoch 13/20	0.02s	Loss: 21.493406	ACC: 0.75364393
Epoch 14/20	0.02s	Loss: 21.5843	ACC: 0.75687176
Epoch 15/20	0.02s	Loss: 21.375723	ACC: 0.76131064
Epoch 16/20	0.02s	Loss: 21.371252	ACC: 0.7601499
Epoch 17/20	0.02s	Loss: 21.160835	ACC: 0.76447487
Epoch 18/20	0.02s	Loss: 21.463665	ACC: 0.7616816
Epoch 19/20	0.02s	Loss: 21.498943	ACC: 0.75785756
Epoch 20/20	0.02s	Loss: 21.439146	ACC: 0.75879306
Trainin

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.78      0.93      0.85      1049
         pos       0.67      0.36      0.47       425

    accuracy                           0.76      1474
   macro avg       0.72      0.64      0.66      1474
weighted avg       0.75      0.76      0.74      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.015)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
log_files = os.listdir(f'./{log_folder}')

for file in log_files:
    if file.startswith("GenericSVMClassifierApproach"):
        approach_log_file=file

with open(f"./{log_folder}/{approach_log_file}") as log_file:
    print(log_file.read())

Training 25 epochs
Epoch 1/25	0.08s	Loss: 28.428364	ACC: 0.70145804
Epoch 2/25	0.02s	Loss: 25.222578	ACC: 0.7186175
Epoch 3/25	0.02s	Loss: 24.759394	ACC: 0.7253143
Epoch 4/25	0.02s	Loss: 24.874647	ACC: 0.7358379
Epoch 5/25	0.02s	Loss: 24.174372	ACC: 0.7493666
Epoch 6/25	0.02s	Loss: 24.102083	ACC: 0.75114214
Epoch 7/25	0.02s	Loss: 23.90326	ACC: 0.75826037
Epoch 8/25	0.02s	Loss: 23.718916	ACC: 0.7595033
Epoch 9/25	0.02s	Loss: 23.759663	ACC: 0.75884074
Epoch 10/25	0.02s	Loss: 23.65178	ACC: 0.7599564
Epoch 11/25	0.02s	Loss: 23.939157	ACC: 0.7549663
Epoch 12/25	0.02s	Loss: 23.699596	ACC: 0.76500756
Epoch 13/25	0.02s	Loss: 23.40783	ACC: 0.7631498
Epoch 14/25	0.02s	Loss: 23.716967	ACC: 0.7654766
Epoch 15/25	0.02s	Loss: 23.703037	ACC: 0.76508707
Epoch 16/25	0.02s	Loss: 23.362638	ACC: 0.7655084
Epoch 17/25	0.02s	Loss: 23.395699	ACC: 0.7700295
Epoch 18/25	0.02s	Loss: 23.764606	ACC: 0.7584538
Epoch 19/25	0.02s	Loss: 23.535133	ACC: 0.763232
Epoch 20/25	0.02s	Loss: 23.614628	ACC: 0.76164985
Epoch 2

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.77      0.94      0.85      1049
         pos       0.69      0.32      0.43       425

    accuracy                           0.76      1474
   macro avg       0.73      0.63      0.64      1474
weighted avg       0.75      0.76      0.73      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")\
        .setEnableInMemoryStorage(True)

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
    ])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A 16-year-old girl with erosive, polyarticular JRA showe...|     neg|[{sentence_embeddings, 0, 129,  A 16-year-old girl with e...|
| A 42-year-old woman had uneventful bilateral laser-assis...|     neg|[{sentence_embeddings, 0, 112,  A 42-year-old woman had u...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows


In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A search of the literature yielded 15 instances of sever...|     neg|[{sentence_embeddings, 0, 174,  A search of the literatur...|
| BACKGROUND: Uterine malignant mixed mesodermal tumor is ...|     neg|[{sentence_embeddings, 0, 89,  BACKGROUND: Uterine malign...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows


In [ ]:
# log_folder = "s3://<bucket>/logs/ADE_logs_healthcare_200d"  #alternative

log_folder="ADE_logs_healthcare_200d"

### ClassifierDL

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(16)\
        .setMaxEpochs(30)\
        .setLr(0.001)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(
    stages = [
        classifier_dl
    ])

In [ ]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.89      0.90      0.89      1049
         pos       0.75      0.72      0.73       425

    accuracy                           0.85      1474
   macro avg       0.82      0.81      0.81      1474
weighted avg       0.85      0.85      0.85      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### MultiClassifierDL

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(32)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']    983
['pos']    491
Name: result, dtype: int64


In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

neg    983
pos    491
Name: result, dtype: int64


In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.91      0.85      0.88      1049
         pos       0.69      0.80      0.74       425

    accuracy                           0.84      1474
   macro avg       0.80      0.82      0.81      1474
weighted avg       0.85      0.84      0.84      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [ ]:
#from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(40)\
    .setBatchSize(16)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.96      0.71      0.81      1049
         pos       0.56      0.92      0.70       425

    accuracy                           0.77      1474
   macro avg       0.76      0.81      0.75      1474
weighted avg       0.84      0.77      0.78      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.81      0.94      0.87      1049
         pos       0.76      0.44      0.56       425

    accuracy                           0.80      1474
   macro avg       0.78      0.69      0.71      1474
weighted avg       0.79      0.80      0.78      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.005)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.81      0.93      0.87      1049
         pos       0.73      0.44      0.55       425

    accuracy                           0.79      1474
   macro avg       0.77      0.69      0.71      1474
weighted avg       0.78      0.79      0.78      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

## Bert Sentence Embeddings (sbiobert_base_cased_mli)

Now we will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = nlp.Pipeline(
    stages = [document_assembler,
              bert_sent])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A 16-year-old girl with erosive, polyarticular JRA showe...|     neg|[{sentence_embeddings, 0, 129,  A 16-year-old girl with e...|
| A 42-year-old woman had uneventful bilateral laser-assis...|     neg|[{sentence_embeddings, 0, 112,  A 42-year-old woman had u...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows


In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A search of the literature yielded 15 instances of sever...|     neg|[{sentence_embeddings, 0, 174,  A search of the literatur...|
| BACKGROUND: Uterine malignant mixed mesodermal tumor is ...|     neg|[{sentence_embeddings, 0, 89,  BACKGROUND: Uterine malign...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows


In [ ]:
log_folder="ADE_logs_bert"

### ClassifierDL

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(32)\
        .setMaxEpochs(20)\
        .setLr(0.001)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.91      0.93      0.92      1049
         pos       0.81      0.77      0.79       425

    accuracy                           0.88      1474
   macro avg       0.86      0.85      0.86      1474
weighted avg       0.88      0.88      0.88      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### MultiClassifierDL

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(32)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_bert = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_bert.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']           1018
['pos']            455
['neg', 'pos']       1
Name: result, dtype: int64


In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

neg    1018
pos     456
Name: result, dtype: int64


In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.92      0.89      0.91      1049
         pos       0.75      0.81      0.78       425

    accuracy                           0.87      1474
   macro avg       0.84      0.85      0.84      1474
weighted avg       0.87      0.87      0.87      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_bert.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.92      0.92      0.92      1049
         pos       0.80      0.79      0.80       425

    accuracy                           0.88      1474
   macro avg       0.86      0.86      0.86      1474
weighted avg       0.88      0.88      0.88      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.88      0.95      0.92      1049
         pos       0.85      0.69      0.76       425

    accuracy                           0.88      1474
   macro avg       0.87      0.82      0.84      1474
weighted avg       0.87      0.88      0.87      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.005)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.92      0.93      0.92      1049
         pos       0.81      0.79      0.80       425

    accuracy                           0.89      1474
   macro avg       0.86      0.86      0.86      1474
weighted avg       0.89      0.89      0.89      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

## DocumentLogRegClassifier

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

normalizer = nlp.Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = nlp.StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = nlp.Stemmer() \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("stem")

logreg = medical.DocumentLogRegClassifierApproach()\
    .setInputCols("stem")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(3)\
    .setTol(1e-6)

clf_Pipeline = nlp.Pipeline(stages=[
    document_assembler,
    tokenizer,
    normalizer,
    stopwords_cleaner,
    stemmer,
    logreg])
doclogreg_model = clf_Pipeline.fit(trainingData)

In [ ]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.87      0.89      0.88      1049
         pos       0.71      0.66      0.69       425

    accuracy                           0.82      1474
   macro avg       0.79      0.78      0.78      1474
weighted avg       0.82      0.82      0.82      1474


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["DocumentLogRegClassifier"] = [res["pos"]["f1-score"], res["accuracy"]]

## Comparision of All Models' Result in ADE Dataset

Here is the all results of classifier models. Only positive ADE f1 scores and accuracy scores are presented.

In [ ]:
results_df

                          pos-f1-score  accuracy
ClassifierDL_100d             0.724311  0.850746
MultiClassifierDL_100d        0.695150  0.820896
GenericClassifier_100d        0.652850  0.727273
GenericLogReg_100d            0.465544  0.763229
GenericSVM_100d               0.434783  0.761872
ClassifierDL_200d             0.733572  0.848711
MultiClassifierDL_200d        0.737991  0.837178
GenericClassifier_200d        0.696889  0.768657
GenericLogReg_200d            0.556548  0.797829
GenericSVM_200d               0.553441  0.793080
ClassifierDL_bert             0.792271  0.883311
MultiClassifierDL_bert        0.778661  0.867707
GenericClassifier_bert        0.797153  0.883989
GenericLogReg_bert            0.760417  0.875170
GenericSVM_bert               0.799523  0.886024
DocumentLogRegClassifier      0.686131  0.824966


# Mtsamples Dataset

## Load Dataset

In [ ]:
# Sample data should be downloaded and uploaded to the s3 bucket

#!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mtsamples_classifier.csv

In [ ]:
spark_df = spark.read.csv(f"{bucket}/mtsamples_classifier.csv", header = True)

spark_df.show(10,truncate=100)

+----------------+----------------------------------------------------------------------------------------------------+
|        category|                                                                                                text|
+----------------+----------------------------------------------------------------------------------------------------+
|Gastroenterology| PROCEDURES PERFORMED: Colonoscopy. INDICATIONS: Renewed symptoms likely consistent with active f...|
|Gastroenterology| OPERATION 1. Ivor-Lewis esophagogastrectomy. 2. Feeding jejunostomy. 3. Placement of two right-s...|
|Gastroenterology| PREOPERATIVE DIAGNOSES: 1. Gastroesophageal reflux disease. 2. Chronic dyspepsia. POSTOPERATIVE ...|
|Gastroenterology| PROCEDURE: Colonoscopy. PREOPERATIVE DIAGNOSES: Rectal bleeding and perirectal abscess. POSTOPER...|
|Gastroenterology| PREOPERATIVE DIAGNOSIS: Right colon tumor. POSTOPERATIVE DIAGNOSES: 1. Right colon cancer. 2. As...|
|Gastroenterology| PREOPERATIVE DIAGNOSI

In [ ]:
spark_df.printSchema()

root
 |-- category: string (nullable = true)
 |-- text: string (nullable = true)


In [ ]:
spark_df.count()

638


In [ ]:
spark_df.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|Gastroenterology|  157|
|       Neurology|  143|
|         Urology|  115|
|      Orthopedic|  223|
+----------------+-----+


In [ ]:
(trainingData, testData) = spark_df.randomSplit([0.8, 0.2], seed = 42)

In [ ]:
trainingData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|Gastroenterology|  132|
|       Neurology|  121|
|         Urology|   95|
|      Orthopedic|  188|
+----------------+-----+


In [ ]:
testData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|Gastroenterology|   25|
|       Neurology|   22|
|         Urology|   20|
|      Orthopedic|   35|
+----------------+-----+


## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



We will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")\
        .setEnableInMemoryStorage(True)

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows


In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows


In [ ]:
testData_with_embeddings.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)


In [ ]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                          sentence_embeddings.embeddings|
+------------------------------------------------------------------------------------------------------------------------+
|[[-0.010433162, 0.0127568655, 0.110687375, 0.1609855, -0.14818177, -0.050856464, 0.064748034, -0.019294674, -0.002178...|
|[[-0.0055608996, 0.018396137, 0.12305377, 0.13861515, -0.18651922, -0.07559964, 0.12978752, -0.0046452526, -0.0031408...|
|[[-0.019461514, 0.060639214, 0.11142021, 0.12826534, -0.16216077, -0.10416892, 0.17136306, -0.020363769, 0.01169337, ...|
+------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows


In [ ]:
log_folder="Mt_logs_healthcare_100d"

### ClassifierDL

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(50)\
        .setLr(0.005)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)\
        # .setValidationSplit(0.1)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.84      0.84      0.84        25
       Neurology       0.80      0.73      0.76        22
      Orthopedic       0.89      0.94      0.92        35
         Urology       0.70      0.70      0.70        20

        accuracy                           0.82       102
       macro avg       0.81      0.80      0.80       102
    weighted avg       0.82      0.82      0.82       102


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df = pd.DataFrame(columns=["macro-f1-score","weighted-f1-score","accuracy"])
results_df.loc["ClassifierDL_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]
results_df

                   macro-f1-score  weighted-f1-score  accuracy
ClassifierDL_100d        0.804643           0.822012  0.823529


### MultiClassifierDL

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(8)\
  .setMaxEpochs(40)\
  .setLr(5e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']                     30
['Gastroenterology']               20
['Urology']                        20
['Neurology']                      20
[]                                  8
['Neurology', 'Urology']            2
['Gastroenterology', 'Urology']     1
['Neurology', 'Orthopedic']         1
Name: result, dtype: int64


In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

Orthopedic          31
Neurology           22
Urology             21
Gastroenterology    20
Name: result, dtype: int64


In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.95      0.90      0.93        21
       Neurology       0.82      0.90      0.86        20
      Orthopedic       0.97      0.91      0.94        33
         Urology       0.90      0.95      0.93        20

        accuracy                           0.91        94
       macro avg       0.91      0.92      0.91        94
    weighted avg       0.92      0.91      0.92        94


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(50)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(False)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_100d.pb


In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.88      0.92        25
       Neurology       0.61      0.86      0.72        22
      Orthopedic       0.93      0.74      0.83        35
         Urology       0.90      0.90      0.90        20

        accuracy                           0.83       102
       macro avg       0.85      0.85      0.84       102
    weighted avg       0.86      0.83      0.84       102


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.91      0.84      0.87        25
       Neurology       0.81      0.77      0.79        22
      Orthopedic       0.84      0.91      0.88        35
         Urology       0.85      0.85      0.85        20

        accuracy                           0.85       102
       macro avg       0.85      0.84      0.85       102
    weighted avg       0.85      0.85      0.85       102


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.88      0.90        25
       Neurology       0.81      0.77      0.79        22
      Orthopedic       0.86      0.91      0.89        35
         Urology       0.85      0.85      0.85        20

        accuracy                           0.86       102
       macro avg       0.86      0.85      0.86       102
    weighted avg       0.86      0.86      0.86       102


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
    ])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows


In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows


In [ ]:
log_folder="Mt_logs_healthcare_200d"

### ClassifierDL

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(50)\
        .setLr(0.005)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.79      0.76      0.78        25
       Neurology       0.88      0.68      0.77        22
      Orthopedic       0.86      0.91      0.89        35
         Urology       0.62      0.75      0.68        20

        accuracy                           0.79       102
       macro avg       0.79      0.78      0.78       102
    weighted avg       0.80      0.79      0.79       102


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### MultiClassifierDL



In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(16)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.2)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']                        32
['Gastroenterology']                  21
['Urology']                           19
['Neurology']                         17
['Neurology', 'Orthopedic']            6
[]                                     2
['Gastroenterology', 'Neurology']      1
['Gastroenterology', 'Orthopedic']     1
['Gastroenterology', 'Urology']        1
['Neurology', 'Urology']               1
['Urology', 'Orthopedic']              1
Name: result, dtype: int64


In [ ]:
# We will get the highest score label as result. you can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

Orthopedic          37
Gastroenterology    23
Urology             21
Neurology           19
Name: result, dtype: int64


In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.91      0.88      0.89        24
       Neurology       0.89      0.81      0.85        21
      Orthopedic       0.89      0.94      0.92        35
         Urology       0.86      0.90      0.88        20

        accuracy                           0.89       100
       macro avg       0.89      0.88      0.88       100
    weighted avg       0.89      0.89      0.89       100


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       1.00      0.80      0.89        25
       Neurology       0.78      0.82      0.80        22
      Orthopedic       0.94      0.86      0.90        35
         Urology       0.74      1.00      0.85        20

        accuracy                           0.86       102
       macro avg       0.87      0.87      0.86       102
    weighted avg       0.88      0.86      0.86       102


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.84      0.84      0.84        25
       Neurology       0.74      0.77      0.76        22
      Orthopedic       0.88      0.86      0.87        35
         Urology       0.80      0.80      0.80        20

        accuracy                           0.82       102
       macro avg       0.82      0.82      0.82       102
    weighted avg       0.82      0.82      0.82       102


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.91      0.84      0.87        25
       Neurology       0.85      0.77      0.81        22
      Orthopedic       0.89      0.91      0.90        35
         Urology       0.78      0.90      0.84        20

        accuracy                           0.86       102
       macro avg       0.86      0.86      0.86       102
    weighted avg       0.87      0.86      0.86       102


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## Bert Sentence Embeddings (sbiobert_base_cased_mli)

We will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = nlp.Pipeline(
    stages = [document_assembler,
              bert_sent])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows


In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows


In [ ]:
log_folder="Mt_logs_bert"

### ClassifierDL

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(30)\
        .setLr(0.002)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.88      0.92        25
       Neurology       0.83      0.86      0.84        22
      Orthopedic       0.88      0.86      0.87        35
         Urology       0.86      0.95      0.90        20

        accuracy                           0.88       102
       macro avg       0.88      0.89      0.88       102
    weighted avg       0.88      0.88      0.88       102


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### MultiClassifierDL

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(8)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_bert = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_bert.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']                 34
['Gastroenterology']           23
['Urology']                    20
['Neurology']                  18
[]                              5
['Neurology', 'Orthopedic']     2
Name: result, dtype: int64


In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

Orthopedic          36
Gastroenterology    23
Urology             20
Neurology           18
Name: result, dtype: int64


In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       1.00      0.92      0.96        25
       Neurology       0.89      0.76      0.82        21
      Orthopedic       0.86      0.91      0.89        34
         Urology       0.85      1.00      0.92        17

        accuracy                           0.90        97
       macro avg       0.90      0.90      0.90        97
    weighted avg       0.90      0.90      0.90        97


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(16)\
    .setLearningRate(0.002)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_bert.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.85      0.88      0.86        25
       Neurology       0.65      0.91      0.75        22
      Orthopedic       0.90      0.77      0.83        35
         Urology       0.93      0.70      0.80        20

        accuracy                           0.81       102
       macro avg       0.83      0.82      0.81       102
    weighted avg       0.84      0.81      0.82       102


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(64)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.92      0.94        25
       Neurology       0.76      0.73      0.74        22
      Orthopedic       0.86      0.86      0.86        35
         Urology       0.82      0.90      0.86        20

        accuracy                           0.85       102
       macro avg       0.85      0.85      0.85       102
    weighted avg       0.85      0.85      0.85       102


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericCLogReg_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(64)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.88      0.92        25
       Neurology       0.77      0.91      0.83        22
      Orthopedic       0.94      0.83      0.88        35
         Urology       0.86      0.95      0.90        20

        accuracy                           0.88       102
       macro avg       0.88      0.89      0.88       102
    weighted avg       0.89      0.88      0.88       102


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericCSVM_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## DocumentLogRegClassifier

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

logreg = medical.DocumentLogRegClassifierApproach()\
    .setInputCols("token")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(10)\
    .setTol(1e-6)

clf_Pipeline = nlp.Pipeline(stages=[document_assembler,
                                tokenizer,
                                logreg])

doclogreg_model = clf_Pipeline.fit(trainingData)

In [ ]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.88      0.90        25
       Neurology       0.80      0.73      0.76        22
      Orthopedic       0.91      0.89      0.90        35
         Urology       0.79      0.95      0.86        20

        accuracy                           0.86       102
       macro avg       0.86      0.86      0.86       102
    weighted avg       0.87      0.86      0.86       102


In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["DocumentLogRegClassifier"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## Comparision of All Models' Result in Mtsamples Dataset

Here is the all results of classifier models. Macro f1, weighted f1 and accuracy scores are presented.

In [ ]:
results_df

                          macro-f1-score  weighted-f1-score  accuracy
ClassifierDL_100d               0.804643           0.822012  0.823529
MultiClassifierDL_100d          0.912075           0.915748  0.914894
GenericClassifier_100d          0.839761           0.839011  0.833333
GenericLogReg_100d              0.848103           0.852503  0.852941
GenericSVM_100d                 0.856886           0.862308  0.862745
ClassifierDL_200d               0.778862           0.794689  0.794118
MultiClassifierDL_200d          0.884583           0.889411  0.890000
GenericClassifier_200d          0.858869           0.864576  0.862745
GenericLogReg_200d              0.816280           0.824088  0.823529
GenericSVM_200d                 0.855785           0.862529  0.862745
ClassifierDL_bert               0.883860           0.882593  0.882353
MultiClassifierDL_bert          0.895870           0.896134  0.896907
GenericClassifier_bert          0.812058           0.816170  0.813725
GenericCLogReg_bert 

In [ ]:
results_df

,macro-f1-score,weighted-f1-score,accuracy
ClassifierDL_100d,0.708342,0.738724,0.745098
MultiClassifierDL_100d,0.884165,0.885508,0.886598
GenericClassifier_100d,0.879519,0.883000,0.882353
GenericLogReg_100d,0.859291,0.862404,0.862745
GenericSVM_100d,0.877119,0.882451,0.882353
ClassifierDL_200d,0.127737,0.175326,0.343137
MultiClassifierDL_200d,0.906453,0.907969,0.908163
GenericClassifier_200d,0.868552,0.871530,0.872549
GenericLogReg_200d,0.834182,0.842500,0.843137
GenericSVM_200d,0.845665,0.852612,0.852941
